In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from cmdstanpy import CmdStanModel
import pandas as pd
import json
import pickle
import os
import xarray as xr

# Preprocessing data

In [2]:

df_raw = pd.read_csv('data/pilot/raw/20_01_2025/tom_dictator_v2_2025-01-20.csv')
                     
# Replace full stop with underscore in df_raw column names
df_raw.columns = df_raw.columns.str.replace('.', '_')

sessions = [
    'tgvqauld',
    'ta176utg'
]

# Only keep rows where session_id is in sessions and where participant_label is not NaN
df_full = df_raw[df_raw['session_code'].isin(sessions) & df_raw['participant_label'].notna() & (df_raw['participant__current_page_name'] == 'End')]
# Insert 
columns = ['participant_code', 'participant_label', 'session_code',
       
       'player_block_type', 'player_block_idx', 'player_idx_in_block', 'player_trust_cond', 'player_trust_label',
       'player_intention', 'player_interest', 
       'player_certainty', 'player_certainty_val', 'player_rigidity', 'player_rigidity_val',
       'player_k_lvl', 'player_trust_predicted', 'player_alpha_prior',
       'player_alpha_prior_entropy', 'player_alpha', 'player_alpha_entropy',
       'player_beta_prior', 'player_beta_prior_entropy', 'player_beta',
       'player_beta_entropy', 'player_beta_social_prior',
       'player_beta_social_prior_entropy', 'player_beta_social',
       'player_beta_social_entropy', 'player_path_taken',
       'player_perceived_certainty_certain',
       'player_perceived_certainty_uncertain',
       'player_perceived_certainty_immutable', 'player_attention',
       'player_attention_passed', 'player_attention_correct',

       'player_focus_work_high', 'player_focus_work_medium', 'player_focus_work_low',
       'player_strategy_high', 'player_strategy_low', 'player_strategy_medium',

       'subsession_round_number',
       'participant__current_app_name',
       'participant__current_page_name', 
       'player_payoff',
]


df = df_full[columns]

# If player is in column name, remove player_ from column name
df.columns = df.columns.str.replace('player_', '')


df_long = df[(df.block_type == 'test') & (df.trust_cond != 'none')][['participant_code', 'intention', 'certainty', 'rigidity', 'intention', 'trust_cond', 'trust_label', 'path_taken']]

# Get dummies for path taken and concat with df_long
path_taken_dummies = pd.get_dummies(df_long['path_taken'], prefix='', prefix_sep='')
df_long = pd.concat([df_long, path_taken_dummies], axis=1)

df_long['delivered'] = df_long['A'] + df_long['B'] > 0
df_long['path_hidden'] = df_long['B'] + df_long['C'] > 0

# Create path_idx column
path_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
}
df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
df_long['path_idx'] = df_long['path_taken'].map(lambda x: path_dict[x])

# Create certainty_lvl column
certainties_dict = {
    'uncertain': 1,
    'certain': 2,
    'immutable': 3,
}
df_long['certainty_lvl'] = df_long['certainty'].map(lambda x: certainties_dict[x])
df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])

# Create trust_lvl column
trusts_dict = {
    'none': 4,
    'very_low': 0.5,
    'low': 1,
    'slightly_low': 1.5,
    'medium': 2,
    'slightly_high': 2.5,
    'high': 3,
    'very_high': 3.5,
}
df_long['trust_lvl'] = df_long['trust_label'].map(lambda x: trusts_dict[x])
df_long['trust_cond_lvl'] = df_long['trust_cond'].map(lambda x: trusts_dict[x])
df['trust_lvl'] = df['trust_label'].map(lambda x: trusts_dict[x])
df['trust_cond_lvl'] = df['trust_cond'].map(lambda x: trusts_dict[x])


# Assign unique integer to each participant and make sure it is the same for both df and df_long and recoverable
df_long['part_idx'] = pd.Categorical(df_long['participant_code']).codes
df['part_idx'] = pd.Categorical(df['participant_code']).codes


paths_colors = {
    'A': np.array([238, 154, 86])/255,
    'B': np.array([183, 117, 63])/255,
    'C': np.array([50, 114, 169])/255,
    'D': np.array([88, 182, 225])/255,
}

/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_63409/1701408934.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_63409/1701408934.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_63409/1701408934.py:87: SettingWithCopyWarning: 
A value is 

# Construct stan dataset

## ToM models

In [3]:
# Pivot df_wide on block_idx after keeping only path_idx as a column
stan_data = dict()

# Get only test blocks and keep only part_idx, certainty_lvl, trust_lvl, path_idx and block_idx
df_wide = df[df.block_type == 'test'][['part_idx', 'certainty_lvl', 'trust_lvl', 'trust_cond_lvl', 'path_idx', 'block_idx']].dropna()

# Pivot df_wide on block_idx for path_idx
df_path_idx = df_wide[['part_idx', 'path_idx', 'block_idx']].dropna()
df_path_idx = df_path_idx.pivot(index='part_idx', columns='block_idx', values='path_idx')
stan_data['y'] = df_path_idx.to_numpy().tolist()

# Pivot df_wide on block_idx for certainty_lvl
df_certainty = df_wide[['part_idx', 'certainty_lvl', 'block_idx']].dropna()
df_certainty = df_certainty.pivot(index='part_idx', columns='block_idx', values='certainty_lvl')
stan_data['certainties'] = df_certainty.to_numpy().tolist()

# Pivot df_wide on block_idx for trust_lvl
df_trust = df_wide[['part_idx', 'trust_cond_lvl', 'block_idx']].dropna()
df_trust = df_trust.pivot(index='part_idx', columns='block_idx', values='trust_cond_lvl')
stan_data['trusts'] = df_trust.to_numpy().astype(int).tolist()

# Meta data
stan_data['M'], stan_data['N'] = df_path_idx.shape
stan_data['A'] = df_path_idx.nunique().max()
stan_data['C'] = df_certainty.nunique().max()
stan_data['T'] = df_trust.nunique().max()

# Stan data to json
json.dump(stan_data, open('stan_files/tom_model_basic_data.json', 'w'))


## Heuristic feature model

In [4]:
stan_data_heuristic = dict()

df_wide = df[df.block_type == 'explore'][['part_idx', 'certainty_lvl', 'trust_lvl', 'trust_cond_lvl', 'path_idx', 'block_idx', 'idx_in_block']].dropna()

# Pivot df_wide on block_idx after keeping only path_idx as a column
df_path_idx = df_wide[['part_idx', 'path_idx', 'block_idx', 'idx_in_block']].dropna()
df_path_idx = df_path_idx.pivot(index='part_idx', columns=['block_idx', 'idx_in_block'], values='path_idx')
df_path_idx = df_path_idx.unstack(level=0)
df_path_idx = df_path_idx.to_xarray().transpose('part_idx', 'block_idx', 'idx_in_block')
stan_data_heuristic['y'] = df_path_idx.to_numpy().tolist()

# Pivot df_wide on block_idx for certainty_lvl
df_certainty = df_wide[['part_idx', 'certainty_lvl', 'block_idx', 'idx_in_block']].dropna()
df_certainty = df_certainty.pivot(index='part_idx', columns=['block_idx', 'idx_in_block'], values='certainty_lvl')
df_certainty = df_certainty.unstack(level=0)
df_certainty = df_certainty.to_xarray().transpose('part_idx', 'block_idx', 'idx_in_block')
stan_data_heuristic['certainties'] = df_certainty.to_numpy().tolist()

# Pivot df_wide on block_idx for trust_lvl
df_trust = df_wide[['part_idx', 'trust_lvl', 'block_idx', 'idx_in_block']].dropna()
df_trust = df_trust.pivot(index='part_idx', columns=['block_idx', 'idx_in_block'], values='trust_lvl')
df_trust = df_trust.unstack(level=0)
df_trust = df_trust.to_xarray().transpose('part_idx', 'block_idx', 'idx_in_block')
stan_data_heuristic['trusts'] = df_trust.to_numpy().tolist()

# Meta data
stan_data_heuristic['M'], stan_data_heuristic['N'], stan_data_heuristic['K'] = df_path_idx.shape
stan_data_heuristic['A'] = np.unique(df_path_idx.to_numpy().flatten()).size
stan_data_heuristic['C'] = np.unique(df_certainty.to_numpy().flatten()).size
stan_data_heuristic['T'] = np.unique(df_trust.to_numpy().flatten()).size

# Test data
df_test = df[df.block_type == 'test'][['part_idx', 'certainty_lvl', 'trust_cond_lvl', 'path_idx', 'block_idx', 'idx_in_block']].dropna()
# Pivot df_test on block_idx after keeping only path_idx as a column
df_test_path_idx = df_test[['part_idx', 'path_idx', 'block_idx']].dropna()
df_test_path_idx = df_test_path_idx.pivot(index='part_idx', columns='block_idx', values='path_idx')
stan_data_heuristic['y_test'] = df_test_path_idx.to_numpy().tolist()
# Pivot df_test on block_idx for certainty_lvl
df_test_certainty = df_test[['part_idx', 'certainty_lvl', 'block_idx']].dropna()
df_test_certainty = df_test_certainty.pivot(index='part_idx', columns='block_idx', values='certainty_lvl')
stan_data_heuristic['certainties_test'] = df_test_certainty.to_numpy().tolist()
# Pivot df_test on block_idx for trust_lvl
df_test_trust = df_test[['part_idx', 'trust_cond_lvl', 'block_idx']].dropna()
df_test_trust = df_test_trust.pivot(index='part_idx', columns='block_idx', values='trust_cond_lvl')
stan_data_heuristic['trusts_test'] = df_test_trust.to_numpy().astype(int).tolist()

# Meta data
_, stan_data_heuristic['N_test'] = df_test_path_idx.shape

# Stan data to json
json.dump(stan_data_heuristic, open('stan_files/heuristic_model_data.json', 'w'))

In [13]:
stan_data_heuristic['certainties']

[[[1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [1, 1, 1, 1, 1],
  [2, 2, 2, 2, 2],
  [3, 3, 3, 3, 3],
  [2, 2, 2, 2, 2],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [2, 2, 2, 2, 2],
  [3, 3, 3, 3, 3]],
 [[1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [2, 2, 2, 2, 2],
  [1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2],
  [1, 1, 1, 1, 1]],
 [[1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [3, 3, 3, 3, 3],
  [2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [2, 2, 2, 2, 2],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3]],
 [[1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [1, 1, 1, 1, 1],
  [2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [2, 2, 2, 2, 2],
  [1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3]],
 [[1, 1, 1, 1, 1],
  [3, 3, 3, 3, 3],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1],
  [2, 2, 2, 2, 2],
  [3, 3, 3, 3, 3],
  [2, 2,

# Compile the model

In [6]:
variants = [
    #'basic_full',
    #'basic_onlyalpha',
    #'basic_nobeta',
    #'basic_nodelta',
    'heuristic_test',
]
variant = variants[0]

stan_file = os.path.join('.', f'tom_model_{variant}.stan')
print(stan_file)
model = CmdStanModel(stan_file=stan_file)

16:19:41 - cmdstanpy - INFO - compiling stan file /Users/vbtesh/Projects/taking_others_for_granted/tom_model_heuristic_test.stan to exe file /Users/vbtesh/Projects/taking_others_for_granted/tom_model_heuristic_test


./tom_model_heuristic_test.stan


16:19:54 - cmdstanpy - INFO - compiled model executable: /Users/vbtesh/Projects/taking_others_for_granted/tom_model_heuristic_test


# Fit the model

In [7]:
if 'heuristic' in variant:
    data_file = os.path.join('.', 'stan_files', 'heuristic_model_data.json')
else:
    data_file = os.path.join('.', 'stan_files', 'tom_model_basic_data.json')

fit = model.sample(
    data=data_file,  
    chains=4,
    parallel_chains=4,
    iter_warmup=2500,
    iter_sampling=1000,
    show_console=False
)

16:19:56 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

16:21:45 - cmdstanpy - INFO - CmdStan done processing.


In [8]:
print(fit.diagnose())

Processing csv files: /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpy3pmva9t/tom_model_heuristic_testt9evom17/tom_model_heuristic_test-20250414161956_1.csv, /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpy3pmva9t/tom_model_heuristic_testt9evom17/tom_model_heuristic_test-20250414161956_2.csv, /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpy3pmva9t/tom_model_heuristic_testt9evom17/tom_model_heuristic_test-20250414161956_3.csv, /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpy3pmva9t/tom_model_heuristic_testt9evom17/tom_model_heuristic_test-20250414161956_4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.



In [9]:
# Pickle the model
with open(f'stan_files/tom_model_{variant}_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Pickle the fit
with open(f'stan_files/tom_model_{variant}_fit.pkl', 'wb') as f:
    pickle.dump(fit, f)

# Inspect the results

In [10]:
df_summary = fit.summary()
quantities = model.generate_quantities(data=data_file, previous_fit=fit)
draws = quantities.draws_pd()
df_summary

12:56:34 - cmdstanpy - INFO - Chain [1] start processing
12:56:34 - cmdstanpy - INFO - Chain [2] start processing
12:56:34 - cmdstanpy - INFO - Chain [3] start processing
12:56:34 - cmdstanpy - INFO - Chain [4] start processing
12:56:35 - cmdstanpy - INFO - Chain [1] done processing
12:56:35 - cmdstanpy - INFO - Chain [2] done processing
12:56:35 - cmdstanpy - INFO - Chain [3] done processing
12:56:35 - cmdstanpy - INFO - Chain [4] done processing
12:56:35 - cmdstanpy - WARNING - Sample doesn't contain draws from warmup iterations, rerun sampler with "save_warmup=True".


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-7472.460000,0.489643,16.120300,-7499.130000,-7472.390000,-7446.30000,1083.890000,26.230400,1.003330
certainty_w[1],-0.011681,0.011311,1.002930,-1.650820,-0.012115,1.63967,7862.010000,190.262000,1.000030
certainty_w[2],0.007785,0.011324,1.001390,-1.660370,0.013655,1.65785,7820.430000,189.256000,0.999174
certainty_w[3],-0.001352,0.010624,1.015370,-1.659940,-0.015673,1.69320,9134.370000,221.053000,0.999190
sigma_certainty_w,0.988426,0.012567,0.983461,0.050243,0.697631,2.95852,6123.883243,148.199101,1.000353
...,...,...,...,...,...,...,...,...,...
"y_sim[99,6]",2.077500,0.016985,1.058670,1.000000,2.000000,4.00000,3885.130000,94.021000,0.999681
"y_sim[99,7]",2.009750,0.016747,1.050680,1.000000,2.000000,4.00000,3936.160000,95.255700,1.000490
"y_sim[99,8]",2.011250,0.016627,1.044700,1.000000,2.000000,4.00000,3947.680000,95.534700,0.999849
"y_sim[99,9]",2.025250,0.015835,1.045410,1.000000,2.000000,4.00000,4358.580000,105.478000,1.000290
